<h1 style='text-align: center;'>Survivor:</h1>
<h2 style='text-align: center;'>Exploring Contestant Trends Through Data</h2>

This projects aims to look at the long-running CBS show Survivor over the years, and determine what demographics of contestants are more likely to win.

We will also be looking to see if there are any trends where certain demographics have lower chances of winning.


In [28]:
import pandas as pd
import matplotlib as plt
import sqlite3


### Importing Data

In [29]:
# I think it will be more difficult than worth to try and get function to create csv if they don't exist
seasons = pd.read_csv('seasons.csv')
contestants = pd.read_csv('contestants.csv')
stats = pd.read_csv('stats.csv')
idols = pd.read_csv('idols.csv')
advantages = pd.read_csv('advantages.csv')
immunities = pd.read_csv('immunities.csv')

### Cleaning Data

In [ ]:
def clean_and_merge():
    # drop columns
    advantages = advantages.drop(columns=['Rank', 'Contestant', 'VV', 'VFB', 'Tie broken?'], )
    idols = idols.drop(columns=['Rank', 'Contestant'], )
    immunities = immunities.drop(columns=['Rank', 'Contstant'], )
    # change column names / strip values (Season column of S, idols table of special characters (*,))
    
    pass # will be used to clean idols, advantages, and immunities. 
# this includes dropping unnecessary columns, stripping values of unnessary charaters, and renaming columns to be more readable
# will likely also use this function to merge all together with stats table, on contestant column(s)
# (i think merging will get rid of duplicate columns the data is merged on. may need to after the fact)
# Also will rename stats table values. after merge?

# will need to create and run a pytest for this to ensure it works correctly?

In [ ]:
# List of Column name meanings (To potentially update to later):
# -- Stats table --
# SurvSc: Survival Score
# SurvAv: Survival Average
# ChW: Challenge Wins
# ChA: Challenge Appearances
# ChW%: Challenge Win %
# SO: Sit Outs
# VFB: Votes For Bootee
# VAP: Votes Against (Total)
# TotV: Total Votes Cast
# TCA: Tribal Council Appearances
# TC%: Tribal Council %
# wTCR: Tribal Council Ratio (weighted)
# JVF: Jury Votes For 
# TotJ: Total Number Of Jurors
# JV%: Jury Votes %
# -- Idols --
# IF: Idols founds
# IH: Idols held
# IP: Idols Played
# VV: Votes voided
# -- Advantages --
# AF: Advantage found
# AP: Advantage played
# VV: Votes voided - redundant (idol list includes advantagess this is relevant to)
# VFB: Votes for booted player (If adv. extra or steal a vote. Will likely drop)
# Tie broken? Only applies to 2 vote block advantages, will drop
# -- Immunity Wins --
# ICW: Individual (immunity) challenge wins
# ICA: Challenge appearances - similar to stats ChW/ChA (but exclusive to individual)

### Exploratory Data Analysis